<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Random_Forest_and_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from tqdm import tqdm

# Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/contract_classification.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())



Mounted at /content/drive


<ipython-input-2-56a844a11275>:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


     ANO_SID CORPORATE_DEVISION   ORTPLZ  ORTS-NAME               STRASSE  \
0  4114028.0                VHV  42109.0  Wuppertal     Hans-Böckler-Str.   
1  4114039.0                VHV  42277.0  Wuppertal       Liegnitzer Str.   
2  4114045.0                VHV  42389.0  Wuppertal             Rascheweg   
3  4114049.0                VGV  42277.0  Wuppertal               Am Diek   
4  4114055.0                VHV  42553.0    Velbert  Emil-Schniewind-Str.   

   SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR        WFL ZONE  ...  \
0      71081.0      NORMAL_VENTURE        1967.565648   69.00000  2.0  ...   
1      55708.0      NORMAL_VENTURE        1967.565648   65.00000  4.0  ...   
2      74148.0      DESIGN_CLASS_I        1967.565648   75.00000  1.0  ...   
3     664000.0      NORMAL_VENTURE        1967.565648  106.24368  NaN  ...   
4      75682.0      NORMAL_VENTURE        1967.565648  119.00000  2.0  ...   

   DRAIN_PIPE_INSURED  PRODUCTLINE  PRIOR_DAMAGES UVV-KZ  UNDERWRITE

In [ ]:
print(contract_class['DAMAGE'].value_counts())

# Assuming your DataFrame is named contract_class
na_counts = contract_class.isna().sum()

print(na_counts)

DAMAGE
0    11389665
1      184774
Name: count, dtype: int64
ANO_SID                     55
CORPORATE_DEVISION           0
ORTPLZ                  102482
ORTS-NAME                 3901
STRASSE                  16338
SUM_INSURED              16503
CONSTRACTION_DESIGN    1188450
CONSTRUCTION_YEAR            0
WFL                      16670
ZONE                    988470
SF-SYSTEM              9717058
TYPE_OF_DEDUCTIBLE           0
DRAIN_PIPE_INSURED           0
PRODUCTLINE            1735746
PRIOR_DAMAGES                0
UVV-KZ                       0
UNDERWRITER                  0
PARTY-ID                   203
contract_year                0
PIPE_PREMIUM_AMOUNT        167
YEAR                         0
DAMAGE                       0
dtype: int64


# Models

## Preprocess the data

In [ ]:
#Choose variables where it makes logical sense

initial_features = contract_class.loc[:, ['CORPORATE_DEVISION','ORTPLZ',  'SUM_INSURED', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED','PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE']]

print(initial_features['DAMAGE'].value_counts())

initial_features_clean = initial_features.dropna()

print(initial_features_clean['DAMAGE'].value_counts())

DAMAGE
0    11389665
1      184774
Name: count, dtype: int64
DAMAGE
0    8516533
1     120574
Name: count, dtype: int64


In [ ]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.head())

    ORTPLZ  SUM_INSURED  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  \
0  42109.0      71081.0        1967.565648   69.0                   0   
1  42277.0      55708.0        1967.565648   65.0                   0   
2  42389.0      74148.0        1967.565648   75.0                   0   
4  42553.0      75682.0        1967.565648  119.0                   0   
5  42113.0      70000.0        1967.565648  100.0                   0   

   PRIOR_DAMAGES  UVV-KZ  DAMAGE  CORPORATE_DEVISION_VHV  \
0              0       1       0                    True   
1              0       1       0                    True   
2              0       1       0                    True   
4              0       1       0                    True   
5              0       1       0                    True   

   CONSTRACTION_DESIGN_CARAVAN_MOTORHOME  ...  ZONE_4  ZONE_4.0  ZONE_5  \
0                                  False  ...   False     False   False   
1                                  False  ...   Fa

## Random Forest

In [ ]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

# Split into test and training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=42)

# Resample the training data
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

In [ ]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# Training the classifier
rf_classifier.fit(X_resampled, y_resampled)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)


In [ ]:
# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))
# With undersampling accuracy gets worse, but we detect more TRUE Positives. Still way more false positive

Accuracy: 0.7853703379950007
Confusion Matrix:
[[1338839  364364]
 [   6392   17827]]
True Positives: 17827
False Positives: 364364
True Negatives: 1338839
False Negatives: 6392
              precision    recall  f1-score   support

           0       1.00      0.79      0.88   1703203
           1       0.05      0.74      0.09     24219

    accuracy                           0.79   1727422
   macro avg       0.52      0.76      0.48   1727422
weighted avg       0.98      0.79      0.87   1727422



In [ ]:
# Feature-importnace analysis
feature_importance = rf_classifier.feature_importances_

# Display results
print("Feature Importance:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {feature_importance[i]}")

Feature Importance:
ORTPLZ: 0.22950934268938258
SUM_INSURED: 0.3436387893280586
CONSTRUCTION_YEAR: 0.10233732052560082
WFL: 0.11902210383701167
DRAIN_PIPE_INSURED: 0.0
PRIOR_DAMAGES: 0.004227427149212484
UVV-KZ: 0.014137294705900449
CORPORATE_DEVISION_VHV: 0.1219592231461422
CONSTRACTION_DESIGN_CARAVAN_MOTORHOME: 8.26432256751105e-06
CONSTRACTION_DESIGN_DESIGN_CLASS_I: 0.00027824805381932284
CONSTRACTION_DESIGN_DESIGN_CLASS_II: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_III: 7.249581611118396e-06
CONSTRACTION_DESIGN_DESIGN_CLASS_IV: 1.330879859145523e-06
CONSTRACTION_DESIGN_DESIGN_CLASS_V: 2.0118167934125968e-07
CONSTRACTION_DESIGN_NORMAL_VENTURE: 0.0010789227884194744
CONSTRACTION_DESIGN_PREDOMINANTLY_WOODEN_ROOF: 0.00014440976673366968
CONSTRACTION_DESIGN_PREFAB_HOUSE: 0.0006388190920825517
CONSTRACTION_DESIGN_PREFAB_HOUSE_I: 0.0
PRODUCTLINE_Kompakt: 0.0061697423506620365
PRODUCTLINE_Plus: 0.001953922569924123
PRODUCTLINE_Premium: 0.001659283059399923
PRODUCTLINE_Sonst: 0.01468830110913276

## Random Forest with Rolling window

In [ ]:
#Choose variables where it makes logical including year column

initial_features = contract_class.loc[:, ['CORPORATE_DEVISION','ORTPLZ',  'SUM_INSURED', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED','PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE', 'YEAR']]

initial_features_clean = initial_features.dropna()

In [ ]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)


In [ ]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

YEAR_INDEX = features_binary.columns.get_loc('YEAR')
print(YEAR_INDEX)

8


In [ ]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize and train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)
# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")



  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [01:08<10:20, 68.93s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    775271
           1       0.46      0.28      0.35      9736

    accuracy                           0.99    785007
   macro avg       0.73      0.64      0.67    785007
weighted avg       0.98      0.99      0.99    785007

Confusion Matrix for year 2015:
[[772052   3219]
 [  6993   2743]]
True Positives: 2743
False Positives: 3219
True Negatives: 772052
False Negatives: 6993
Classification report for year 2016:


 20%|██        | 2/10 [02:21<09:30, 71.29s/it]

...

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    797092
           1       0.30      0.14      0.19     13298

    accuracy                           0.98    810390
   macro avg       0.64      0.57      0.59    810390
weighted avg       0.97      0.98      0.98    810390

Confusion Matrix for year 2022:
[[792603   4489]
 [ 11415   1883]]
True Positives: 1883
False Positives: 4489
True Negatives: 792603
False Negatives: 11415
Classification report for year 2023:


 90%|█████████ | 9/10 [11:25<01:20, 80.13s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    800711
           1       0.26      0.11      0.15     11149

    accuracy                           0.98    811860
   macro avg       0.62      0.55      0.57    811860
weighted avg       0.98      0.98      0.98    811860

Confusion Matrix for year 2023:
[[797130   3581]
 [  9921   1228]]
True Positives: 1228
False Positives: 3581
True Negatives: 797130
False Negatives: 9921
Classification report for year 2024:


100%|██████████| 10/10 [12:46<00:00, 76.70s/it]

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    806672
           1       0.04      0.10      0.06      2049

    accuracy                           0.99    808721
   macro avg       0.52      0.55      0.53    808721
weighted avg       1.00      0.99      0.99    808721

Confusion Matrix for year 2024:
[[801513   5159]
 [  1841    208]]
True Positives: 208
False Positives: 5159
True Negatives: 801513
False Negatives: 1841
Year: 2015, Recall Score: 0.2817378800328677
Year: 2016, Recall Score: 0.24003698566805362
Year: 2017, Recall Score: 0.23204895305478535
Year: 2018, Recall Score: 0.15367132867132868
Year: 2019, Recall Score: 0.1657051282051282
Year: 2020, Recall Score: 0.16186700055648304
Year: 2021, Recall Score: 0.1511731135066582
Year: 2022, Recall Score: 0.1416002406376899
Year: 2023, Recall Score: 0.11014440757018566
Year: 2024, Recall Score: 0.10151293313811616


## Random Forrest with expanding window

In [ ]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'].isin(train_years)]
    y_train = y[features_binary['YEAR'].isin(train_years)]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize and train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [00:15<02:16, 15.17s/it]

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    775271
           1       0.39      0.29      0.33      9736

    accuracy                           0.99    785007
   macro avg       0.69      0.64      0.66    785007
weighted avg       0.98      0.99      0.98    785007

Confusion Matrix for year 2015:
[[770816   4455]
 [  6922   2814]]
True Positives: 2814
False Positives: 4455
True Negatives: 770816
False Negatives: 6922
Classification report for year 2016:


 20%|██        | 2/10 [00:44<03:09, 23.71s/it]

...

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    797092
           1       0.24      0.15      0.19     13298

    accuracy                           0.98    810390
   macro avg       0.61      0.57      0.59    810390
weighted avg       0.97      0.98      0.98    810390

Confusion Matrix for year 2022:
[[790580   6512]
 [ 11275   2023]]
True Positives: 2023
False Positives: 6512
True Negatives: 790580
False Negatives: 11275
Classification report for year 2023:


 90%|█████████ | 9/10 [13:32<02:14, 134.95s/it]

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    800711
           1       0.23      0.12      0.16     11149

    accuracy                           0.98    811860
   macro avg       0.61      0.56      0.58    811860
weighted avg       0.98      0.98      0.98    811860

Confusion Matrix for year 2023:
[[796005   4706]
 [  9761   1388]]
True Positives: 1388
False Positives: 4706
True Negatives: 796005
False Negatives: 9761
Classification report for year 2024:


100%|██████████| 10/10 [17:01<00:00, 102.15s/it]

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    806672
           1       0.04      0.12      0.06      2049

    accuracy                           0.99    808721
   macro avg       0.52      0.55      0.53    808721
weighted avg       1.00      0.99      0.99    808721

Confusion Matrix for year 2024:
[[800968   5704]
 [  1811    238]]
True Positives: 238
False Positives: 5704
True Negatives: 800968
False Negatives: 1811
Year: 2015, Recall Score: 0.2890304026294166
Year: 2016, Recall Score: 0.24539990753582985
Year: 2017, Recall Score: 0.2355865761545081
Year: 2018, Recall Score: 0.15795454545454546
Year: 2019, Recall Score: 0.16979166666666667
Year: 2020, Recall Score: 0.16805787423483584
Year: 2021, Recall Score: 0.15668991756499684
Year: 2022, Recall Score: 0.15212813956986013
Year: 2023, Recall Score: 0.1244954704457799
Year: 2024, Recall Score: 0.1161542215714983


# Naive Bayes

In [ ]:
# Define initial features
initial_features = contract_class.loc[:, ['CORPORATE_DEVISION', 'SUM_INSURED', 'CONSTRACTION_DESIGN', 'CONSTRUCTION_YEAR', 'WFL', 'ZONE', 'DRAIN_PIPE_INSURED', 'PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER', 'DAMAGE', 'YEAR']]
initial_features_clean = initial_features.dropna()

# Define feature and target
columns_to_encode = ['CORPORATE_DEVISION', 'CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create binary variables using one-hot encoding
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separate continuous and discrete features
continuous_features = ['SUM_INSURED', 'CONSTRUCTION_YEAR']
# The discrete features need to be adjusted after one-hot encoding
discrete_features = list(set(X.columns) - set(continuous_features))

In [ ]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

# Train GaussianNB on continuous features
    gnb = GaussianNB()
    gnb.fit(X_train[continuous_features], y_train)

# Train MultinomialNB on discrete features
    mnb = MultinomialNB()
    mnb.fit(X_train[discrete_features], y_train)

    # Get predicted probabilities for continuous and discrete features
    gnb_prob = gnb.predict_proba(X_test[continuous_features])[:, 1]
    mnb_prob = mnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (gnb_prob + mnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)
# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [00:03<00:27,  3.02s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    775271
           1       0.42      0.12      0.19      9736

    accuracy                           0.99    785007
   macro avg       0.70      0.56      0.59    785007
weighted avg       0.98      0.99      0.98    785007

Confusion Matrix for year 2015:
[[773616   1655]
 [  8536   1200]]
True Positives: 1200
False Positives: 1655
True Negatives: 773616
False Negatives: 8536
Classification report for year 2016:


 20%|██        | 2/10 [00:06<00:26,  3.28s/it]

...

              precision    recall  f1-score   support

           0       0.98      0.98      0.98    797092
           1       0.04      0.05      0.05     13298

    accuracy                           0.96    810390
   macro avg       0.51      0.52      0.51    810390
weighted avg       0.97      0.96      0.97    810390

Confusion Matrix for year 2022:
[[780519  16573]
 [ 12590    708]]
True Positives: 708
False Positives: 16573
True Negatives: 780519
False Negatives: 12590
Classification report for year 2023:


 90%|█████████ | 9/10 [00:27<00:03,  3.11s/it]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98    800711
           1       0.04      0.07      0.05     11149

    accuracy                           0.96    811860
   macro avg       0.51      0.52      0.51    811860
weighted avg       0.97      0.96      0.97    811860

Confusion Matrix for year 2023:
[[779298  21413]
 [ 10356    793]]
True Positives: 793
False Positives: 21413
True Negatives: 779298
False Negatives: 10356
Classification report for year 2024:


100%|██████████| 10/10 [00:30<00:00,  3.08s/it]

              precision    recall  f1-score   support

           0       1.00      0.96      0.98    806672
           1       0.01      0.08      0.01      2049

    accuracy                           0.96    808721
   macro avg       0.50      0.52      0.49    808721
weighted avg       1.00      0.96      0.98    808721

Confusion Matrix for year 2024:
[[775821  30851]
 [  1877    172]]
True Positives: 172
False Positives: 30851
True Negatives: 775821
False Negatives: 1877
Year: 2015, Recall Score: 0.12325390304026294
Year: 2016, Recall Score: 0.10864539990753583
Year: 2017, Recall Score: 0.09599388086815183
Year: 2018, Recall Score: 0.029195804195804195
Year: 2019, Recall Score: 0.030208333333333334
Year: 2020, Recall Score: 0.03311074012242626
Year: 2021, Recall Score: 0.03690551680405834
Year: 2022, Recall Score: 0.0532410888855467
Year: 2023, Recall Score: 0.07112745537716388
Year: 2024, Recall Score: 0.08394338701805759


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

Accuracy: 0.99

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99   1719308
           1       0.98      0.04      0.08     24428

    accuracy                           0.99   1743736
   macro avg       0.98      0.52      0.54   1743736
weighted avg       0.99      0.99      0.98   1743736


Confusion Matrix:
 [[1719284      24]
 [  23369    1059]]
True Positives: 1059
False Positives: 24
True Negatives: 1719284
False Negatives: 23369
